### Extract a single column containing free-text feedback and run sentiment analysis on it

Import pandas, set filename, load contents from file from CSV into dataframe, print dataframe shape to verify load.

In [ ]:
import pandas as pd
filename = '~/Downloads/AWS Summit London 2018_May 22, 2018_04.25.csv'

df = pd.read_csv(filename,index_col=False, header=0,low_memory=False)
print("Data loaded into pandas dataframe with shape: {}".format(df.shape))


Extract the desired column from the dataframe by name. Drop empty values at the same time.

In [ ]:
column_name = 'Q17'
feedback = df.dropna(subset=[column_name])[column_name]

Print sample of 5 records. Note it's a dataframe, so it comes with keys and metadata. 

In [ ]:
print feedback.sample(n=5)

Import modules. 
* boto3 is our AWS SDK. 
* operator allows us to sort dictionary objects creating lists. 
* sys gives us some line printing utility

Define a function to allow us to overprint one line

In [ ]:
import boto3
import operator
import sys

def restart_line():
    sys.stdout.write('\r')
    sys.stdout.flush()

Initialise variables

Do the work.
* Create a comprehend client
* itterate over the contents of feedback
* send each line to comprehend.detect_sentiment and get results
* increment the counter on the highest probability sentiment
* if it's a positive, increment positives
* increment the record count
* print status
* once done itterating, print the results

In [ ]:
count = 0
positives = []
scoredict = {
                "Neutral" : 0,
                "Positive" : 0,
                "Negative" :0,
                "Mixed" : 0
            }

comprehendClient = boto3.client('comprehend')

for i in feedback.sample(n=5):
    comprehendClientResponse = comprehendClient.detect_sentiment(
        Text=i,
        LanguageCode='en')
    scores = comprehendClientResponse['SentimentScore']
    sorted_scores = sorted(scores.items(), key=operator.itemgetter(1))
    
    scoredict[sorted_scores[3][0]] += 1 
    count = count +1
    if sorted_scores[3][0] == 'Positive':
        positives.append(sorted_scores[3][0])

    sys.stdout.write("Processed {} records. Positives: {}".format(count,len(positives)))
    sys.stdout.flush()
    restart_line()
        
print("\nTotal number of records: {}.\n".format(count))

for result in sorted(scoredict.items(), 
                     reverse=True, 
                     key=operator.itemgetter(1)):
    print("{} : {}".format(result[1],result[0]))